In [2]:
from obspy.io.segy.segy import SEGYFile

from set_notebook_path import check_path
check_path()

'C:\\Users\\dkb73\\Desktop\\Fullstack\\terean-full-stack'

In [41]:
import numpy as np
import pandas as pd
import re
import obspy
import segyio

In [58]:
sgy_path = "SampleData/GeomSGY/0078.sgy"

In [59]:
def parse_trace_headers(segyfile, n_traces):
    '''
    Parse the segy file trace headers into a pandas dataframe.
    Column names are defined from segyio internal tracefield
    One row per trace
    '''
    # Get all header keys
    headers = segyio.tracefield.keys
    # Initialize dataframe with trace id as index and headers as columns
    df = pd.DataFrame(index=range(1, n_traces + 1),
                      columns=headers.keys())
    # Fill dataframe with all header values
    for k, v in headers.items():
        df[k] = segyfile.attributes(v)[:]
    return df

def parse_text_header(segyfile):
    '''
    Format segy text header into a readable, clean dict
    '''
    raw_header = segyio.tools.wrap(segyfile.text[0])
    # Cut on C*int pattern
    cut_header = re.split(r'C ', raw_header)[1::]
    # Remove end of line return
    text_header = [x.replace('\n', ' ') for x in cut_header]
    text_header[-1] = text_header[-1][:-2]
    # Format in dict
    clean_header = {}
    i = 1
    for item in text_header:
        key = "C" + str(i).rjust(2, '0')
        i += 1
        clean_header[key] = item
    return clean_header


In [60]:
with segyio.open(sgy_path, ignore_geometry=True) as f:
    # Get basic attributes
    n_traces = f.tracecount
    sample_rate = segyio.tools.dt(f) / 1000
    n_samples = f.samples.size
    twt = f.samples
    data = f.trace.raw[:]  # Get all data into memory (could cause on big files)
    # Load headers
    bin_headers = f.bin
    text_headers = parse_text_header(f)
    trace_headers = parse_trace_headers(f, n_traces)
f'N Traces: {n_traces}, N Samples: {n_samples}, Sample rate: {sample_rate}ms'


'N Traces: 24, N Samples: 15000, Sample rate: 2.0ms'

In [61]:
trace_headers

,TRACE_SEQUENCE_LINE,TRACE_SEQUENCE_FILE,FieldRecord,TraceNumber,EnergySourcePoint,CDP,CDP_TRACE,TraceIdentificationCode,NSummedTraces,NStackedTraces,...,TraceIdentifier,ScalarTraceHeader,SourceType,SourceEnergyDirectionMantissa,SourceEnergyDirectionExponent,SourceMeasurementMantissa,SourceMeasurementExponent,SourceMeasurementUnit,UnassignedInt1,UnassignedInt2
1,1,1,78,1,0,0,1,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
2,2,2,78,2,0,0,2,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
3,3,3,78,3,0,0,3,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
4,4,4,78,4,0,0,4,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
5,5,5,78,5,0,0,5,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
6,6,6,78,6,0,0,6,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
7,7,7,78,7,0,0,7,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
8,8,8,78,8,0,0,8,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
9,9,9,78,9,0,0,9,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128
10,10,10,78,10,0,0,10,1,0,0,...,2013,1,0,3292,-5333,-1203173406,-1209,0,0,398000128


In [68]:
np.abs(trace_headers['ElevationScalar'])

1     100
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
11    100
12    100
13    100
14    100
15    100
16    100
17    100
18    100
19    100
20    100
21    100
22    100
23    100
24    100
Name: ElevationScalar, dtype: int32

In [70]:
elevation = trace_headers['ReceiverGroupElevation'] / (-1 * trace_headers['ElevationScalar'])
elevation

1     123.12
2    -456.00
3     789.00
4     121.00
5     467.00
6     432.00
7     793.00
8     204.00
9     302.00
10    342.00
11    349.00
12    509.00
13    292.00
14    304.00
15    567.00
16    234.00
17    444.00
18    667.00
19      1.00
20      1.00
21      1.00
22      1.00
23      1.00
24      1.00
dtype: float64

In [66]:
x = trace_headers['GroupX'] / (-1 * trace_headers['SourceGroupScalar'])
x

1      951.0
2     -177.0
3     1252.0
4        1.1
5       14.0
6       78.0
7       15.0
8       15.0
9      111.0
10      90.0
11     100.0
12     110.0
13     120.0
14     130.0
15     140.0
16     150.0
17     160.0
18     170.0
19     180.0
20     190.0
21     200.0
22     210.0
23     220.0
24     230.0
dtype: float64

In [67]:
y = trace_headers['GroupY'] / (-1 * trace_headers['SourceGroupScalar'])
y

1     0.0
2     1.0
3     0.0
4     2.0
5     0.0
6     3.0
7     0.0
8     4.0
9     0.0
10    5.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [73]:
x_points = x
y_points = y
z_points = elevation
test = [
        {
            "index": idx,
            "x": float(x_points.iloc[idx]),
            "y": float(y_points.iloc[idx]),
            "z": float(z_points.iloc[idx]),
        } for idx in range(len(x_points))
    ]
test

[{'index': 0, 'x': 951.0, 'y': 0.0, 'z': 123.12},
 {'index': 1, 'x': -177.0, 'y': 1.0, 'z': -456.0},
 {'index': 2, 'x': 1252.0, 'y': 0.0, 'z': 789.0},
 {'index': 3, 'x': 1.1, 'y': 2.0, 'z': 121.0},
 {'index': 4, 'x': 14.0, 'y': 0.0, 'z': 467.0},
 {'index': 5, 'x': 78.0, 'y': 3.0, 'z': 432.0},
 {'index': 6, 'x': 15.0, 'y': 0.0, 'z': 793.0},
 {'index': 7, 'x': 15.0, 'y': 4.0, 'z': 204.0},
 {'index': 8, 'x': 111.0, 'y': 0.0, 'z': 302.0},
 {'index': 9, 'x': 90.0, 'y': 5.0, 'z': 342.0},
 {'index': 10, 'x': 100.0, 'y': 0.0, 'z': 349.0},
 {'index': 11, 'x': 110.0, 'y': 0.0, 'z': 509.0},
 {'index': 12, 'x': 120.0, 'y': 0.0, 'z': 292.0},
 {'index': 13, 'x': 130.0, 'y': 0.0, 'z': 304.0},
 {'index': 14, 'x': 140.0, 'y': 0.0, 'z': 567.0},
 {'index': 15, 'x': 150.0, 'y': 0.0, 'z': 234.0},
 {'index': 16, 'x': 160.0, 'y': 0.0, 'z': 444.0},
 {'index': 17, 'x': 170.0, 'y': 0.0, 'z': 667.0},
 {'index': 18, 'x': 180.0, 'y': 0.0, 'z': 1.0},
 {'index': 19, 'x': 190.0, 'y': 0.0, 'z': 1.0},
 {'index': 20, 'x

In [35]:
sgy_read = obspy.read(sgy_path, dtype=np.float64)

In [36]:
sgy_read

24 Trace(s) in Stream:

Seq. No. in line:    1 | 2025-02-17T08:20:30.000000Z - 2025-02-17T08:20:59.998000Z | 500.0 Hz, 15000 samples
...
(22 other traces)
...
Seq. No. in line:   24 | 2025-02-17T08:20:30.000000Z - 2025-02-17T08:20:59.998000Z | 500.0 Hz, 15000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [57]:
idx = 1
print(sgy_read[idx].stats['segy']['trace_header']['receiver_group_elevation'])
print(sgy_read[idx].stats['segy']['trace_header']['surface_elevation_at_source'])
print(sgy_read[idx].stats['segy']['trace_header']['source_depth_below_surface'])
print(sgy_read[idx].stats['segy']['trace_header']['datum_elevation_at_receiver_group'])
print(sgy_read[idx].stats['segy']['trace_header']['datum_elevation_at_source'])
print(sgy_read[idx].stats['segy']['trace_header']['x_coordinate_of_ensemble_position_of_this_trace'])
print(sgy_read[idx].stats['segy']['trace_header']['y_coordinate_of_ensemble_position_of_this_trace'])
print(sgy_read[idx].stats['segy']['trace_header']['source_coordinate_x'])
print(sgy_read[idx].stats['segy']['trace_header']['source_coordinate_y'])
print(sgy_read[idx].stats['segy']['trace_header']['group_coordinate_x'])
print(sgy_read[idx].stats['segy']['trace_header']['group_coordinate_y'])

100
0
0
0
0
0
0
0
0
1000
0


In [7]:
type(sgy_read)

obspy.core.stream.Stream

In [9]:
from obspy.io.segy.segy import SEGYFile

In [14]:
with open(sgy_path, 'rb') as f:
    sgy_file = SEGYFile(file=f, unpack_headers=True)

In [18]:
sgy_file.textual_file_header

b'C 1 CLIENT                        COMPANY                       CREW NO         C 2 LINE            AREA                        MAP ID                          C 3 REEL NO           DAY START OF REEL     YEAR         OBSERVER               C 4 INSTRUMENT: MFG            MODEL            SERIAL NO                       C 5 DATA TRACES/RECORD        AUXULIARY TRACES/RECORD         CDP FOLD          C 6 SAMPLE INTERVAL         SAMPLES/TRACE       BITS/IN      BYTES/SAMPLE       C 7 RECORDING FORMAT        FORMAT THIS REEL        MESUREMENT SYSTEM           C 8 SAMPLE CODE: FLOATING PT     FIXED PT     FIXED PT-GAIN     CORRELATED      C 9 GAIN  TYPE: FIXED     BINARY     FLOATING POINT     OTHER                   C10 FILTERS: ALIAS     HZ  NOTCH     HZ  BAND     -     HZ  SLOPE    -    DB/OCTC11 SOURCE: TYPE            NUMBER /POINT       POINT INTERVAL                  C12 PATTERN:                               LENGTH        WIDTH                  C13 SWEEP: START     HZ  END     HZ  L